In [1]:
%load_ext autoreload
from coffea import hist, util

import coffea.processor as processor
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
import awkward as ak
import numpy as np
import uproot
import re

from pprint import pprint
import matplotlib.pyplot as plt

In [2]:
%autoreload 2
import sys 
import os

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
from ttgenv.Utils.genParentage import maxHistoryPDGID

In [6]:
class PhotonSelector(processor.ProcessorABC):
    def __init__(self, isMC=False):
        # In the initializer, any of the outputs you would like to produce are defined (ex. histograms)
        # Coffea histograms are defined in the same way as in the previous exercise
        # define a list of axes first
        
        dataset_axis = hist.Cat("dataset","Dataset")

        #Declare axis 
        #m3_axis = hist.Bin("M3", r"$M_3$ [GeV]", 40, 0., 1000)
        mass_axis = hist.Bin("mass", r"$m_{\ell\gamma}$ [GeV]", 40, -10., 10)
        pt_axis = hist.Bin("pt", r"$p_{T}$ [GeV]", 40, 0., 400)
        eta_axis = hist.Bin("eta", r"$\eta_{\gamma}$", 50, -2.5, 2.5)
        #chIso_axis = hist.Bin("chIso", r"Charged Hadron Isolation", np.arange(-0.1,20.001,.05))
        
        relIso_axis=hist.Bin("relIso", "relIso", 50,-4.5,4.5)
        phi_axis = hist.Bin("phi","$\phi$", 64, -3.2, 3.2)
        
        
        #Define the accumulator object, a dictionary storing all of the histograms and counters 
        #that we will fill later in the process function
        self.isMC = isMC

        self._accumulator = processor.dict_accumulator({
            
        
            'photon_pt':processor.column_accumulator(np.ndarray(shape=(0,))),
           
        
            'photon_eta':processor.column_accumulator(np.ndarray(shape=(0,))),
            'photon_phi':processor.column_accumulator(np.ndarray(shape=(0,))),
            'label':processor.column_accumulator(np.ndarray(shape=(0,))),

            'charge':processor.column_accumulator(np.ndarray(shape=(0,))),
            'cleanmask':processor.column_accumulator(np.ndarray(shape=(0,))),
            'cutBased':processor.column_accumulator(np.ndarray(shape=(0,))),
            'cutBased17Bitmap':processor.column_accumulator(np.ndarray(shape=(0,))),
            'eCorr':processor.column_accumulator(np.ndarray(shape=(0,))),
            'electronIdx':processor.column_accumulator(np.ndarray(shape=(0,))),
            'electronIdxG':processor.column_accumulator(np.ndarray(shape=(0,))),
            'electronVeto':processor.column_accumulator(np.ndarray(shape=(0,))),
            'energyErr':processor.column_accumulator(np.ndarray(shape=(0,))),
            'genPartFlav':processor.column_accumulator(np.ndarray(shape=(0,))),
            'genPartIdx':processor.column_accumulator(np.ndarray(shape=(0,))),
            'genPartIdxG':processor.column_accumulator(np.ndarray(shape=(0,))),
            'hoe':processor.column_accumulator(np.ndarray(shape=(0,))),
            'isScEtaEB':processor.column_accumulator(np.ndarray(shape=(0,))),
            'isScEtaEE':processor.column_accumulator(np.ndarray(shape=(0,))),
            'jetIdx':processor.column_accumulator(np.ndarray(shape=(0,))),
            'jetIdxG':processor.column_accumulator(np.ndarray(shape=(0,))),
            'mass':processor.column_accumulator(np.ndarray(shape=(0,))),
            'mvaID':processor.column_accumulator(np.ndarray(shape=(0,))),
            'mvaID17':processor.column_accumulator(np.ndarray(shape=(0,))),
            'mvaID17_WP80':processor.column_accumulator(np.ndarray(shape=(0,))),
            'mvaID17_WP90':processor.column_accumulator(np.ndarray(shape=(0,))),
            'mvaID_WP80':processor.column_accumulator(np.ndarray(shape=(0,))),
            'mvaID_WP90':processor.column_accumulator(np.ndarray(shape=(0,))),
            'pdgId':processor.column_accumulator(np.ndarray(shape=(0,))),
            
            'pfRelIso03_all':processor.column_accumulator(np.ndarray(shape=(0,))),
            'pfRelIso03_chg':processor.column_accumulator(np.ndarray(shape=(0,))),
            
            'pixelSeed':processor.column_accumulator(np.ndarray(shape=(0,))),
            'r9':processor.column_accumulator(np.ndarray(shape=(0,))),
            'seedGain':processor.column_accumulator(np.ndarray(shape=(0,))),
            'sieie':processor.column_accumulator(np.ndarray(shape=(0,))),
            'vidNestedWPBitmap':processor.column_accumulator(np.ndarray(shape=(0,))),
          
            'no_jets':processor.column_accumulator(np.ndarray(shape=(0,))),
            #'no_pho':processor.column_accumulator(np.ndarray(shape=(0,))),
             
            
            'phoJetDR':processor.column_accumulator(np.ndarray(shape=(0,))),
            'phoMuDR':processor.column_accumulator(np.ndarray(shape=(0,))),
            #'phoEleDR':processor.column_accumulator(np.ndarray(shape=(0,))),

            'jetMuDR':processor.column_accumulator(np.ndarray(shape=(0,))),
            #'jetEleDR':processor.column_accumulator(np.ndarray(shape=(0,))),
            'jetPhoDR':processor.column_accumulator(np.ndarray(shape=(0,))),

            
            
            'EventCount': processor.value_accumulator(int),

        }
        )
        

    @property
    def accumulator(self):
        return self._accumulator

    

    # The process method is where the heart of the analysis is.  
    # This is where all of the selections are done and the histograms get filled 
    #  (things you did in notebook cells before will be done here instead)
    def process(self, events):
        ### The process function is where most of the work happens. As we'll see below, this is
        ### where the main analysis work happens (object cuts, event selections, filling histograms). 
        
        output = self.accumulator.identity()
        output['EventCount'] = len(events)

        dataset = events.metadata['dataset']
        
        if self.isMC:
            idx = ak.to_numpy(ak.flatten(abs(events.GenPart.pdgId)))
            par = ak.to_numpy(ak.flatten(events.GenPart.genPartIdxMother))
            num = ak.to_numpy(ak.num(events.GenPart.pdgId))        
            maxParentFlatten = maxHistoryPDGID(idx,par,num)
            events["GenPart","maxParent"] = ak.unflatten(maxParentFlatten, num)

        
        ######################
        ###OverlapREMOVAL#####
        ######################
        
        doOverlapRemoval = False
        if 'TTbar' in dataset:
            doOverlapRemoval = True
            overlapPt = 10.
            overlapEta = 5.
            overlapDR = 0.1
        if re.search("^W+[1234]jets$", dataset):
            doOverlapRemoval = True
            overlapPt = 10.
            overlapEta = 2.5
            overlapDR = 0.05


            
        if doOverlapRemoval:
            genmotherIdx = events.GenPart.genPartIdxMother
            genpdgid = events.GenPart.pdgId

            #potential overlap photons are only those passing the kinematic cuts 
            overlapPhoSelect = ((events.GenPart.pt>=overlapPt) & 
                                (abs(events.GenPart.eta) < overlapEta) & 
                                (events.GenPart.pdgId==22) & 
                                (events.GenPart.status==1) & 
                                (events.GenPart.maxParent < 37)
                               )
            overlapPhotons = events.GenPart[overlapPhoSelect] 

            #also require that photons are separate from all other gen particles
            finalGen = events.GenPart[((events.GenPart.status==1)|(events.GenPart.status==71)) & (events.GenPart.pt > 0.01) &
                                      ~((abs(events.GenPart.pdgId)==12) | (abs(events.GenPart.pdgId)==14) | (abs(events.GenPart.pdgId)==16)) &
                                      ~overlapPhoSelect]

            #calculate dR between overlap photons and nearest gen particle
            phoGen, phoGenDR = overlapPhotons.nearest(finalGen, return_metric = True)
            phoGenMask = ak.fill_none(phoGenDR > overlapDR, True)

            #the event is overlapping with the separate sample if there is an overlap photon passing the dR cut, kinematic cuts, and not coming from hadronic activity
            isOverlap = ak.any(phoGenMask, axis=-1)
            passOverlapRemoval = ~isOverlap

        else:
            passOverlapRemoval = np.ones_like(len(events))==1
        
        
        ######################
        ###Object Selection###
        ######################
        
        #Add Tight Muon Select
        muons=events.Muon
        muonSelectTight = ((muons.pt>30) &
                           (abs(muons.eta)<2.4) &
                           (muons.tightId) &
                           (muons.pfRelIso04_all < 0.15)
                          )
        #Add Jet Select
        jets=events.Jet
        jetSelectTight = ((jets.pt>30) &
                          (abs(jets.eta)<2.4) &
                          (jets.isTight)
                         )

        #Add b-tagged Jet Select
        btaggedJetSelect = (jetSelectTight &
                           (jets.btagDeepB>0.6321)
                          )
        
        #Add Electron Select
        electrons=events.Electron
        electronSelectTight = ((electrons.pt> 35) &
                           (abs(electrons.eta)<2.1) &
                            electrons.cutBased>=4   
                         )
        #Add Photon Select
        photon=events.Photon
        photonSelect= ((photon.pt>20) & 
                        (abs(photon.eta) < 1.4442) &
                        (photon.isScEtaEE | photon.isScEtaEB) &
                        (photon.electronVeto) & 
                        np.invert(photon.pixelSeed) 
                       )
        #photonID= photon.cutBased >=2
        #Apply Selection
        tightMuons = muons[muonSelectTight]
        tightJets = jets[jetSelectTight]
        tightBJets = jets[btaggedJetSelect]
        photons = photon[photonSelect]
        tightPho = photon[photonSelect]
        #tightPho =photon[(photonID & photonSelect)]
        tightEle = electrons[electronSelectTight]
        
        ###### dR #######
        phoMu, phoMuDR  =tightPho.nearest(tightMuons,return_metric=True)
        phoEle, phoEleDR = tightPho.nearest(tightEle, return_metric=True)
        phoJet, phoJetDR  = tightPho.nearest(tightJets,return_metric=True)
        
        ##check dR jet,lepton & jet,photon
        jetMu, jetMuDR = tightJets.nearest(tightMuons, return_metric=True)
        jetEle, jetEleDR = tightJets.nearest(tightEle, return_metric=True)
        jetPho, jetPhoDR = tightJets.nearest(tightPho, return_metric=True)
                                                             
        #print('DR:',phoMuDR)
        #print('PT:',tightPho.pt)
        
        #####################
        ###Event Selection###
        #####################
        
        #Apply trigger and add event selection 
        trigger = events.HLT.IsoMu24 | events.HLT.IsoTkMu24
        
        one_muon=(ak.num(tightMuons)==1)
        four_jets=(ak.num(tightJets)>=4)
        btag_jet=(ak.num(tightBJets)>=1)
        zero_ele=(ak.num(tightEle)==0)
        one_photon=(ak.num(tightPho)==1)
        
        # Select events passing the trigger, with exactly one tight muon, one tight photon, no electrons,  ≥4 jets, and ≥ 1 b-tagged jets. 
        eventSelection = (trigger &
                          one_muon &
                          four_jets & 
                          btag_jet &
                          one_photon &
                          zero_ele)
        
        #E=events[eventSelection]
        #leading_pho=tightPho[:,:1]
        #Photon=leading_pho[eventSelection]
        
        #print('After:', phoMuDR[eventSelection])
         
        
        #return output
        pt=processor.column_accumulator(np.array(ak.to_numpy(ak.flatten(tightPho.pt[:,:1][eventSelection]))))
        output['photon_pt']+=pt
        
        eta=processor.column_accumulator(np.array(ak.to_numpy(ak.flatten(tightPho.eta[:,:1][eventSelection]))))
        output['photon_eta']+=eta
        
        phi=processor.column_accumulator(np.array(ak.to_numpy(ak.flatten(tightPho.phi[:,:1][eventSelection]))))
        output['photon_phi']+=phi
          
        charge=processor.column_accumulator(np.array(ak.to_numpy(ak.flatten(tightPho.charge[:,:1][eventSelection]))))
        output['charge']+=charge
        
        
        cleanmask=processor.column_accumulator(np.array(ak.to_numpy(ak.flatten(tightPho.cleanmask[:,:1][eventSelection]))))
        output['cleanmask']+=cleanmask

        
        cutBased=processor.column_accumulator(np.array(ak.to_numpy(ak.flatten(tightPho.cutBased[:,:1][eventSelection]))))
        output['cutBased']+=cutBased

        
        cutBased17Bitmap=processor.column_accumulator(np.array(ak.to_numpy(ak.flatten(tightPho.cutBased17Bitmap[:,:1][eventSelection]))))
        output['cutBased17Bitmap']+=cutBased17Bitmap

        
        eCorr=processor.column_accumulator(np.array(ak.to_numpy(ak.flatten(tightPho.eCorr[:,:1][eventSelection]))))
        output['eCorr']+=eCorr
    
        
        electronIdx=processor.column_accumulator(np.array(ak.to_numpy(ak.flatten(tightPho.electronIdx[:,:1][eventSelection]))))
        output['electronIdx']+=electronIdx

        
        electronIdxG=processor.column_accumulator(np.array(ak.to_numpy(ak.flatten(tightPho.electronIdxG[:,:1][eventSelection]))))
        output['electronIdxG']+=electronIdxG

        
        electronVeto=processor.column_accumulator(np.array(ak.to_numpy(ak.flatten(tightPho.electronVeto[:,:1][eventSelection]))))
        output['electronVeto']+=electronVeto

        
        energyErr=processor.column_accumulator(np.array(ak.to_numpy(ak.flatten(tightPho.energyErr[:,:1][eventSelection]))))
        output['energyErr']+=energyErr

        if self.isMC:
            
            genPartFlav=processor.column_accumulator(np.array(ak.to_numpy(ak.flatten(tightPho.genPartFlav[:,:1][eventSelection]))))
            output['genPartFlav']+=genPartFlav

        
            genPartIdx=processor.column_accumulator(np.array(ak.to_numpy(ak.flatten(tightPho.genPartIdx[:,:1][eventSelection]))))
            output['genPartIdx']+=genPartIdx

        
            genPartIdxG=processor.column_accumulator(np.array(ak.to_numpy(ak.flatten(tightPho.genPartIdxG[:,:1][eventSelection]))))
            output['genPartIdxG']+=genPartIdxG

        
        hoe=processor.column_accumulator(np.array(ak.to_numpy(ak.flatten(tightPho.hoe[:,:1][eventSelection]))))
        output['hoe']+=hoe

        
        isScEtaEB=processor.column_accumulator(np.array(ak.to_numpy(ak.flatten(tightPho.isScEtaEB[:,:1][eventSelection]))))
        output['isScEtaEB']+=isScEtaEB

        
        isScEtaEE=processor.column_accumulator(np.array(ak.to_numpy(ak.flatten(tightPho.isScEtaEE[:,:1][eventSelection]))))
        output['isScEtaEE']+=isScEtaEE

        
        jetIdx=processor.column_accumulator(np.array(ak.to_numpy(ak.flatten(tightPho.jetIdx[:,:1][eventSelection]))))
        output['jetIdx']+=jetIdx

        
        jetIdxG=processor.column_accumulator(np.array(ak.to_numpy(ak.flatten(tightPho.jetIdxG[:,:1][eventSelection]))))
        output['jetIdxG']+=jetIdxG

        
        mvaID=processor.column_accumulator(np.array(ak.to_numpy(ak.flatten(tightPho.mvaID[:,:1][eventSelection]))))
        output['mvaID']+=mvaID

        
        mvaID17=processor.column_accumulator(np.array(ak.to_numpy(ak.flatten(tightPho.mvaID17[:,:1][eventSelection]))))
        output['mvaID17']+=mvaID17

        
        mvaID17_WP80=processor.column_accumulator(np.array(ak.to_numpy(ak.flatten(tightPho.mvaID17_WP80[:,:1][eventSelection]))))
        output['mvaID17_WP80']+=mvaID17_WP80

        
        mvaID17_WP90=processor.column_accumulator(np.array(ak.to_numpy(ak.flatten(tightPho.mvaID17_WP90[:,:1][eventSelection]))))
        output['mvaID17_WP90']+=mvaID17_WP90

        
        mvaID_WP80=processor.column_accumulator(np.array(ak.to_numpy(ak.flatten(tightPho.mvaID_WP80[:,:1][eventSelection]))))
        output['mvaID_WP80']+=mvaID_WP80

        
        mvaID_WP90=processor.column_accumulator(np.array(ak.to_numpy(ak.flatten(tightPho.mvaID_WP90[:,:1][eventSelection]))))
        output['mvaID_WP90']+=mvaID_WP90

        
        pdgId=processor.column_accumulator(np.array(ak.to_numpy(ak.flatten(tightPho.pdgId[:,:1][eventSelection]))))
        output['pdgId']+=pdgId

            
        pfRelIso03_all=processor.column_accumulator(np.array(ak.to_numpy(ak.flatten(tightPho.pfRelIso03_all[:,:1][eventSelection]))))
        output['pfRelIso03_all']+=pfRelIso03_all

        
        pfRelIso03_chg=processor.column_accumulator(np.array(ak.to_numpy(ak.flatten(tightPho.pfRelIso03_chg[:,:1][eventSelection]))))
        output['pfRelIso03_chg']+=pfRelIso03_chg
   
        pixelSeed=processor.column_accumulator(np.array(ak.to_numpy(ak.flatten(tightPho.pixelSeed[:,:1][eventSelection]))))
        output['pixelSeed']+=pixelSeed

        
        r9=processor.column_accumulator(np.array(ak.to_numpy(ak.flatten(tightPho.r9[:,:1][eventSelection]))))
        output['r9']+=r9

        
        seedGain=processor.column_accumulator(np.array(ak.to_numpy(ak.flatten(tightPho.seedGain[:,:1][eventSelection]))))
        output['seedGain']+=seedGain

        
        sieie=processor.column_accumulator(np.array(ak.to_numpy(ak.flatten(tightPho.sieie[:,:1][eventSelection]))))
        output['sieie']+=sieie

        
        #print((np.array(ak.to_numpy(ak.flatten(phoMuDR[eventSelection])))))
        
        phoMuDR=processor.column_accumulator(np.array(ak.to_numpy(ak.flatten(phoMuDR[eventSelection]))))
        output['phoMuDR']+=phoMuDR
                                                             
        #phoEleDR=processor.column_accumulator(np.array(ak.flatten(phoEleDR[eventSelection])))
        #output['phoEleDR']+=phoEleDR  
        #print((np.array(ak.to_numpy(ak.flatten(phoJetDR[eventSelection])))))
        
        #print((ak.to_numpy(ak.flatten(phoJetDR[eventSelection]))))
        #print((ak.flatten(phoJetDR[eventSelection])))
        #print((phoJetDR[eventSelection]))
        #print(phoJetDR)
        
        #print(type(np.array(ak.to_numpy(ak.flatten(phoJetDR[eventSelection])))))
        #print(type(ak.to_numpy(ak.flatten(phoJetDR[eventSelection]))))
        #print(type(ak.flatten(phoJetDR[eventSelection])))
        
        phoJetDR=processor.column_accumulator(np.array(ak.to_numpy(ak.flatten(phoJetDR[eventSelection]))))
        output['phoJetDR']+=phoJetDR
                                                       
        jetMuDR=processor.column_accumulator(np.array(ak.to_numpy(ak.flatten(jetMuDR[eventSelection][:,:1]))))
        output['jetMuDR']+=jetMuDR
                                                             
        #jetEleDR=processor.column_accumulator(np.array(ak.flatten(jetEleDR[eventSelection])))
        #output['jetEleDR']+=jetEleDR
        
        jetPhoDR=processor.column_accumulator(np.array(ak.to_numpy(ak.flatten(jetPhoDR[eventSelection][:,:1]))))
        output['jetPhoDR']+=jetPhoDR                                                   
                                                             
        vidNestedWPBitmap=processor.column_accumulator(np.array(ak.to_numpy(ak.flatten(tightPho.vidNestedWPBitmap[:,:1][eventSelection])))) 
        output['vidNestedWPBitmap']+=vidNestedWPBitmap
        
        jet_no=processor.column_accumulator(np.array(ak.to_numpy(ak.num(tightJets)>=4)))
        output['no_jets']+=jet_no
        
        
        
        if dataset=='TTGamma':
            output['label']+=processor.column_accumulator(np.ones_like(ak.to_numpy(ak.flatten(tightPho.pt[:,:1][eventSelection]))))
        elif dataset=='TTbar':
            output['label']+=processor.column_accumulator(np.zeros_like(ak.to_numpy(ak.flatten(tightPho.pt[:,:1][eventSelection]))))
        else:
            output['label']+=processor.column_accumulator(-1*np.ones_like(ak.to_numpy(ak.flatten(tightPho.pt[:,:1][eventSelection]))))

            
        return output
    
    def postprocess(self, accumulator):
        return accumulator

In [40]:
#Define files to run over
skimDir="root://cmseos.fnal.gov//store/user/lpctop/TTGamma_FullRun2/Skims_v6-2/2016/"

fileset = {#"TTGamma":[f"{skimDir}/TTGamma_SingleLept_2016_skim.root"],
           "TTbar":[#f"{skimDir}/TTbarPowheg_Semilept_2016_skim_1of10.root",
                    #f"{skimDir}/TTbarPowheg_Semilept_2016_skim_2of10.root",
                    #f"{skimDir}/TTbarPowheg_Semilept_2016_skim_3of10.root",
                    #f"{skimDir}/TTbarPowheg_Semilept_2016_skim_4of10.root",
                    #f"{skimDir}/TTbarPowheg_Semilept_2016_skim_5of10.root",
                    #f"{skimDir}/TTbarPowheg_Semilept_2016_skim_6of10.root",
                    #f"{skimDir}/TTbarPowheg_Semilept_2016_skim_7of10.root",
                    #f"{skimDir}/TTbarPowheg_Semilept_2016_skim_8of10.root",
                    #f"{skimDir}/TTbarPowheg_Semilept_2016_skim_9of10.root",
                    f"{skimDir}/TTbarPowheg_Semilept_2016_skim_10of10.root"
                   ],
#           "WGamma":[f"{skimDir}/WGamma_2016_skim.root"],
#           "Z+jets":[f'{skimDir}/DYjetsM50_ext2_2016_skim_1of10.root'],
           #"W+3jets":[f"{skimDir}/W3jets_2016_skim.root"],
           #"W+4jets":[f"{skimDir}/W4jets_2016_skim.root"],
          }

#filesetData = {"DataMu":[f"{skimDir}/Data_SingleMu_b_2016_skim_1of10.root", 
#                         f"{skimDir}/Data_SingleMu_b_2016_skim_2of10.root"], 
#              }

In [41]:
np.warnings.filterwarnings('ignore')

#the NanoAODSchema needs to be adjusted, to remove cross references to FSRPhotons
class SkimmedSchema(NanoAODSchema):
    def __init__(self, base_form):
        base_form["contents"].pop("Muon_fsrPhotonIdx", None)
        super().__init__(base_form)

#Run Coffea code using uproot
outputMC = processor.run_uproot_job(
    fileset,  #dictionary of datasets to run on, defined earlier in this cell
    "Events", #Name of the TTree you will be opening
    PhotonSelector(isMC=True),  #Coffea processor you defined
    processor.futures_executor,
    executor_args={"schema": SkimmedSchema,'workers': 2},  ## workers = 2, parallelize jobs, running 2 at once
    chunksize=100000, #in each chunk, use 1 million events
    maxchunks=2, #limit to using only 3 chunks for each dataset (useful for testing purposes)
)

Processing:   0%|          | 0/2 [00:00<?, ?chunk/s]

In [17]:
#outputMC

In [42]:
util.save(outputMC, 'TTBar10_2chunks.coffea')

In [12]:
test=util.load('TTGamma_pt.coffea')

In [14]:
dir(test)

['__abstractmethods__',
 '__add__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__radd__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__slotnames__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_cache',
 '_abc_negative_cache',
 '_abc_negative_cache_version',
 '_abc_registry',
 'add',
 'clear',
 'copy',
 'fromkeys',
 'get',
 'identity',
 'items',
 'keys',
 'pop',
 'popitem',
 'setdefault',
 'update',
 'values']

In [16]:
import coffea
coffea.__version__

'0.7.1'